<a href="https://colab.research.google.com/github/Bharathi3009/Talent-Opportunity-Index/blob/main/TOI_Simple_Mathematical_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd

# Load dataset
df = pd.read_csv('/content/AI_Resume_Screening.csv')

# Display the first few rows of the DataFrame
display(df.head())

,Resume_ID,Name,Skills,Experience (Years),Education,Certifications,Job Role,Recruiter Decision,Salary Expectation ($),Projects Count,AI Score (0-100)
0,1,Ashley Ali,"TensorFlow, NLP, Pytorch",10,B.Sc,NaN,AI Researcher,Hire,104895,8,100
1,2,Wesley Roman,"Deep Learning, Machine Learning, Python, SQL",10,MBA,Google ML,Data Scientist,Hire,113002,1,100
2,3,Corey Sanchez,"Ethical Hacking, Cybersecurity, Linux",1,MBA,Deep Learning Specialization,Cybersecurity Analyst,Hire,71766,7,70
3,4,Elizabeth Carney,"Python, Pytorch, TensorFlow",7,B.Tech,AWS Certified,AI Researcher,Hire,46848,0,95
4,5,Julie Hill,"SQL, React, Java",4,PhD,NaN,Software Engineer,Hire,87441,9,100


In [ ]:
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
import numpy as np

# Simple preprocessing
df['Experience (Years)'] = df['Experience (Years)'].fillna(df['Experience (Years)'].mean())
df['Education'] = df['Education'].fillna('Bachelor')
df['Skills'] = df['Skills'].fillna('')

# Feature engineering
df['num_skills'] = df['Skills'].apply(lambda x: 0 if x.strip() == '' else len(x.split(',')))
df.drop(columns=['Skills'], inplace=True)

edu_map = {'Bachelor': 0, 'Master': 1, 'PhD': 2}
df['education_code'] = df['Education'].map(edu_map)
df.drop(columns=['Education'], inplace=True)

scaler = StandardScaler()
df['experience_norm'] = scaler.fit_transform(df[['Experience (Years)']])
df.drop(columns=['Experience (Years)'], inplace=True)

X = df[['experience_norm', 'num_skills', 'education_code']]
df['Hire'] = df['Recruiter Decision'].map({'Hire': 1, 'Reject': 0})
y = df['Hire']

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [ ]:
# Define simple weights for features (these can be tuned manually)
w_experience = 0.5
w_skills = 0.3
w_education = 0.2

# Calculate TOI as weighted sum of normalized features
def compute_toi(X):
    return (
        X['experience_norm'] * w_experience +
        X['num_skills'] * w_skills +
        X['education_code'] * w_education
    )

# Compute TOI scores for test data
toi_scores = compute_toi(X_test)

# Normalize TOI scores to 0–1 range
toi_scores_norm = (toi_scores - np.min(toi_scores)) / (np.max(toi_scores) - np.min(toi_scores))

# Use threshold of 0.5 to classify Hire vs. Reject
y_pred = (toi_scores_norm >= 0.5).astype(int)

In [ ]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, roc_auc_score

# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
roc_auc = roc_auc_score(y_test, y_pred)

print(f"Accuracy: {accuracy:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"ROC AUC: {roc_auc:.4f}")

Accuracy: 0.2967
Precision: 1.0000
Recall: 0.1059
ROC AUC: 0.5530
